In [1]:
from __future__ import print_function

import os
import sys
import platform

import argparse

import torch
from torch import nn, optim
from torch.nn import functional
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.utils import save_image
from torchvision import datasets, transforms

In [2]:
print("Python Version: {}".format(platform.python_version()))
print("Torch Version: {}".format(torch.__version__))

Python Version: 3.7.1
Torch Version: 1.0.0


# Get MNIST Data

In [3]:
epochs = 5
batchSize = 128
logInterval = 100
imageSize = 28
learning_rate = 1e-3

In [4]:
transform = transforms.Compose([transforms.ToTensor()])
#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

In [ ]:
mnistTrainData = datasets.MNIST("Data", download=True, train=True, transform=transform)
mnistTestData = datasets.MNIST("Data", download=True, train=False, transform=transform)

In [ ]:
print("Number of Training Samples: {}".format(len(mnistTrainData)))
print("Number of Testing Samples: {}".format(len(mnistTestData)))

Number of Training Samples: 60000
Number of Testing Samples: 10000


In [ ]:
trainDataLoader = DataLoader(mnistTrainData, batch_size=batchSize, shuffle=True)
testDataLoader = DataLoader(mnistTestData, batch_size=batchSize, shuffle=False)

# Autoencoder Model

In [ ]:
class Autoencoder(torch.nn.Module):
    def __init__(self, hiddenLayer):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, hiddenLayer))
        self.decoder = nn.Sequential(nn.Linear(hiddenLayer, 28 * 28), torch.nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Train Model

In [ ]:
def Train(epoch):
    model.train()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

    for batch_idx, (data, target) in enumerate(trainDataLoader):
        img = data.view(data.size(0), -1)
        img = Variable(img)

        output = model(img)
        loss = criterion(output, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, epochs, loss.data.item()))


        
def Test(epoch, hiddenLayer):
    model.eval()
    criterion = nn.MSELoss()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(testDataLoader):            
            img = data.view(data.size(0), -1)
            img = Variable(img)

            output = model(img)
            loss = criterion(output, img)

            test_loss += loss.item()
            if i == 0:
                pic = (output.cpu().data).view(-1, 1, 28, 28)
                save_image(pic, 'Data/reconstruction_{}_{}.png'.format(hiddenLayer, epoch))

    test_loss /= len(testDataLoader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

# Undercomplete Autoencoder

In [ ]:
underCompleteHiddenLayer = 64
model = Autoencoder(underCompleteHiddenLayer)
for epoch in range(epochs):
    Train(epoch)
    Test(epoch, underCompleteHiddenLayer)

epoch [1/5], loss:0.0161
====> Test set loss: 0.0001
epoch [2/5], loss:0.0130
====> Test set loss: 0.0001
epoch [3/5], loss:0.0117
====> Test set loss: 0.0001
epoch [4/5], loss:0.0131
====> Test set loss: 0.0001
epoch [5/5], loss:0.0118
====> Test set loss: 0.0001


# Overcomplete Autoencoder

In [ ]:
overCompleteHiddenLayer = 1600
model = Autoencoder(overCompleteHiddenLayer)
for epoch in range(epochs):
    Train(epoch)
    Test(epoch, overCompleteHiddenLayer)

epoch [1/5], loss:0.0037
====> Test set loss: 0.0000
epoch [2/5], loss:0.0033
====> Test set loss: 0.0000
epoch [3/5], loss:0.0034
====> Test set loss: 0.0000
epoch [4/5], loss:0.0034
====> Test set loss: 0.0000
epoch [5/5], loss:0.0033
====> Test set loss: 0.0000


# Visualize Ouput

# Visualize Latent Properties

# Variational Autoencoders

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = functional.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = functional.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = functional.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def Train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(trainDataLoader):
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(trainDataLoader.dataset)))

def Test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(testDataLoader):
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],recon_batch.view(batchSize, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),'Data/reconstruction_{}.png'.format(epoch), nrow=n)

    test_loss /= len(testDataLoader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
torch.manual_seed(1)
model = VAE()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(epochs):
    Train(epoch)
    Test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28), 'Data/sample_' + str(epoch) + '.png')